In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import ast
import operator

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
import sys
from os.path import dirname
sys.path.append(dirname("../../"))

from src.utilities import *
from src.reccomendation_util import *

In [3]:
%load_ext autoreload
%autoreload

Inputs required by the user

In [4]:
pwd

'/Users/ashish1610dhiman/data_projects/fall22_hw/RoboChef/notebooks/recommendation'

In [5]:
# Output from CNN. Input for Recommendation
PREDICTED_LABEL = "apple pie"

Load the Input Data

In [6]:
label_ingr_mapping = pd.read_csv("../../data/recommendation/label_ingredients_mapping.csv")
raw_interactions = pd.read_csv("../../data/recommendation/RAW_interactions.csv")
recipe_metadata = pd.read_csv("../../data/recommendation/RAW_recipes.csv")
recipe_metadata['nutrition_list'] = recipe_metadata['nutrition'].apply(lambda x: ast.literal_eval(x))
recipe_metadata['ingredients_list'] = recipe_metadata['ingredients'].apply(lambda x: ast.literal_eval(x))
recipe_metadata[['calories', 'fat_dv', "sugar_dv", \
                     "sodium_dv", "protein_dv", "sat_fat", "carbs_dv"]\
               ] = pd.DataFrame(recipe_metadata.nutrition_list.tolist(), index=recipe_metadata.index)

Get ingredients for the label

In [7]:
def get_label_ingredients(data, label):
    """
    Function to return ingredients after mapping the label to recipe
    Input : data
    Output : list of ingredients
    """
    ingredients_list = data[data['label'] == label]['ingredients'][0]
    ingredients_list = ast.literal_eval(ingredients_list)
    
    return ingredients_list

### Get user_id for the user

In [8]:
# Create input user id for recommendation prediction
def get_user_id(data):
    user_list = data['user_id'].unique().tolist()
    input_user_id = random.choice(user_list)
    return input_user_id

In [9]:
ingredients_list = get_label_ingredients(label_ingr_mapping, PREDICTED_LABEL)
input_user_id = get_user_id(raw_interactions)

In [10]:
print(input_user_id) 
print(ingredients_list)

539445
['apple juice', 'raw honey', 'whole cloves', 'everclear alcohol', 'ground cinnamon', 'cheesecloth']


### Final Recommendation using SVD

In [15]:
ad_final_reccom(input_user_id,ingredients_list, raw_interactions, recipe_metadata, k= 10)

,name,id,rating_pred,minutes,ingredients_list,nutrition_list
5,fruited irish oatmeal,80652,4.656340,435,"[water, steel cut oats, apple juice, dried cra...","[193.1, 3.0, 49.0, 5.0, 12.0, 2.0, 12.0]"
9,vanilla poached pears,133911,4.651098,70,"[pears, apple juice, vanilla beans, ground cin...","[66.7, 0.0, 45.0, 0.0, 0.0, 0.0, 5.0]"
6,homemade apple pie liquor,235381,4.647718,15,"[apple cider, apple juice, sugar, cinnamon sti...","[967.3, 2.0, 881.0, 11.0, 2.0, 1.0, 81.0]"
4,free range fruitcake,106542,4.647524,1500,"[golden raisin, currants, sun-dried cranberrie...","[460.4, 22.0, 178.0, 22.0, 10.0, 39.0, 22.0]"
2,chicken with apple cranberry and bacon,100651,4.596873,45,"[bacon, flour, ground cinnamon, poultry season...","[517.6, 42.0, 35.0, 16.0, 66.0, 50.0, 8.0]"
7,omani spice mix,492862,4.593053,5,"[ground coriander, ground cumin, ground cinnam...","[66.0, 5.0, 1.0, 0.0, 4.0, 2.0, 3.0]"
1,bircher muesli,96689,4.592650,10,"[rolled oats, apple juice, plain yogurt, curra...","[693.2, 29.0, 192.0, 2.0, 40.0, 21.0, 39.0]"
3,eggnog extraordinaire,261241,4.588004,1440,"[milk, whole cloves, vanilla, cinnamon, egg yo...","[459.2, 47.0, 121.0, 3.0, 16.0, 90.0, 12.0]"
8,tea house chai concentrate,413590,4.581534,20,"[black peppercorns, fresh gingerroot, cardamom...","[15.1, 0.0, 8.0, 0.0, 0.0, 0.0, 1.0]"
0,autumn glow punch,490618,4.575500,40,"[whole allspice, whole cloves, cinnamon stick,...","[44.1, 0.0, 37.0, 0.0, 0.0, 0.0, 3.0]"


In [16]:
ad_final_reccom(input_user_id,ingredients_list, raw_interactions, recipe_metadata, threshold = 0.0, k= 10)

,name,id,rating_pred,minutes,ingredients_list,nutrition_list
5,mexican stack up rsc,486261,4.922170,75,"[poblano pepper, reynolds wrap foil, olive oil...","[793.0, 83.0, 10.0, 34.0, 87.0, 131.0, 11.0]"
3,crunchy valley chicken rsc,495202,4.918118,55,"[hidden valley original ranch dressing mix, sa...","[598.4, 60.0, 26.0, 21.0, 39.0, 62.0, 14.0]"
6,saucy tortellini and meatball soup a1,518069,4.902447,50,"[ground turkey, breadcrumbs, egg, granulated g...","[388.4, 37.0, 14.0, 55.0, 62.0, 35.0, 3.0]"
4,just peachy grillin ribs rsc,487669,4.902435,100,"[racks of baby-back pork ribs, kosher salt, ro...","[903.2, 86.0, 97.0, 76.0, 132.0, 98.0, 11.0]"
1,beef stroganov sauteed beef in sour cream sauce,97243,4.895244,60,"[mustard powder, sugar, salt, vegetable oil, o...","[301.9, 38.0, 43.0, 35.0, 10.0, 50.0, 5.0]"
8,turkey cutlets with balsamic brown sugar sauce,146325,4.894346,35,"[all-purpose flour, dried thyme, salt, black p...","[336.5, 9.0, 37.0, 23.0, 82.0, 6.0, 6.0]"
0,basic vegan chocolate cupcakes,259057,4.891191,40,"[soymilk, apple cider vinegar, granulated suga...","[157.8, 10.0, 53.0, 6.0, 4.0, 5.0, 7.0]"
9,wait family pizza dough bread machine,187415,4.887933,16,"[water, margarine, egg, bread flour, all-purpo...","[529.6, 20.0, 34.0, 22.0, 25.0, 13.0, 29.0]"
2,chilean salsa,63836,4.885785,25,"[garlic cloves, green serrano chilies, sea sal...","[374.3, 43.0, 62.0, 73.0, 11.0, 18.0, 9.0]"
7,sweet and sour stir fry shrimp with broccoli a...,163311,4.880751,30,"[vegetable oil, carrots, onion, red bell peppe...","[440.7, 25.0, 77.0, 85.0, 71.0, 10.0, 12.0]"


In [19]:
protein_mask = recipe_metadata.protein_dv>20
calories_mask = recipe_metadata.calories<600
ad_final_reccom(input_user_id,ingredients_list, raw_interactions, recipe_metadata, threshold = 0.25,\
               extra_filters = [protein_mask,calories_mask], k=10)

,name,id,rating_pred,minutes,ingredients_list,nutrition_list
1,chicken with apple cranberry and bacon,100651,4.596873,45,"[bacon, flour, ground cinnamon, poultry season...","[517.6, 42.0, 35.0, 16.0, 66.0, 50.0, 8.0]"
3,fiesta stew,374693,4.547296,140,"[all-purpose flour, ground cumin, chili powder...","[596.3, 28.0, 84.0, 33.0, 70.0, 29.0, 24.0]"
2,cinnamon honey muesli,110739,4.538237,40,"[rolled oats, skim milk, apple juice, currants...","[406.0, 9.0, 168.0, 3.0, 25.0, 4.0, 26.0]"
5,low fat apple ginger spice whole wheat waffles,301645,4.520757,10,"[applesauce, unbleached all-purpose flour, who...","[345.5, 6.0, 54.0, 9.0, 27.0, 7.0, 21.0]"
8,saffron rice,484514,4.481408,35,"[ghee, blanched almonds, onion, long grain ric...","[560.5, 29.0, 50.0, 12.0, 29.0, 24.0, 28.0]"
9,vanilla matcha chai latte dairy free,478680,4.472549,25,"[water, blanched almond, cardamom pods, pepper...","[524.2, 60.0, 85.0, 1.0, 32.0, 16.0, 11.0]"
6,pinto beans and prunes vegan,497606,4.470316,30,"[dried pink beans, olive oil, yellow onion, ga...","[503.8, 29.0, 65.0, 0.0, 33.0, 13.0, 23.0]"
4,ham with apple raisin sauce,356593,4.456929,110,"[all-purpose flour, oven cooking bag, tart app...","[551.7, 46.0, 75.0, 4.0, 92.0, 55.0, 7.0]"
0,bavarian pot roast,162797,4.456781,310,"[boneless beef top round, apple juice, tomato ...","[301.5, 22.0, 17.0, 31.0, 65.0, 27.0, 2.0]"
7,pork tenderloin saddleback,61505,4.443353,55,"[pork tenderloin, olive oil, kosher salt, grou...","[596.0, 14.0, 281.0, 5.0, 94.0, 14.0, 26.0]"


In [18]:
protein_mask = recipe_metadata.protein_dv>20
calories_mask = recipe_metadata.calories<600
ad_final_reccom(input_user_id,ingredients_list, raw_interactions, recipe_metadata, threshold = 0.25,\
               extra_filters = [protein_mask,calories_mask],k=5)

,name,id,rating_pred,minutes,ingredients_list,nutrition_list
0,chicken with apple cranberry and bacon,100651,4.596873,45,"[bacon, flour, ground cinnamon, poultry season...","[517.6, 42.0, 35.0, 16.0, 66.0, 50.0, 8.0]"
2,fiesta stew,374693,4.547296,140,"[all-purpose flour, ground cumin, chili powder...","[596.3, 28.0, 84.0, 33.0, 70.0, 29.0, 24.0]"
1,cinnamon honey muesli,110739,4.538237,40,"[rolled oats, skim milk, apple juice, currants...","[406.0, 9.0, 168.0, 3.0, 25.0, 4.0, 26.0]"
3,low fat apple ginger spice whole wheat waffles,301645,4.520757,10,"[applesauce, unbleached all-purpose flour, who...","[345.5, 6.0, 54.0, 9.0, 27.0, 7.0, 21.0]"
4,saffron rice,484514,4.481408,35,"[ghee, blanched almonds, onion, long grain ric...","[560.5, 29.0, 50.0, 12.0, 29.0, 24.0, 28.0]"
